In [2]:
# to add the spiking model codebase to the path
import sys
sys.path.append('..')

%matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.animation import FuncAnimation, writers
# plt.style.use('seaborn-pastel')
plt.rcParams['animation.ffmpeg_path'] ='C:\\ffmpeg\\bin\\ffmpeg.exe'

from brian2 import pA, Hz, second, ms
import numpy as np

from results import Results
from plot import (MidpointNormalize, weight_matrix, remove_axes_less, prune_figure, prune_figure_less, prune_figure_more,
                  plot_spike_raster, plot_firing_rate, plot_voltage_detail)
from generate import weight_matrix_from_flat_inds_weights
from stats import moving_average, regress_linear, remove_outliers
import matplotlib.font_manager as fm

Using matplotlib backend: Qt5Agg


INFO       Cache size for target "cython": 8721 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "C:\Users\mikejseay\.cython\brian_extensions" directory. [brian2]


In [58]:
plt.close('all')

In [5]:
targetPath = 'C:/Users/mikejseay/Documents/BrianResults/cross-homeo-pre-outer-explodeDealTest/'
# targetFile = 'classicJercog_1000_0p5_balance_randomUniformSaray_movieTest_2021-05-14-08-22_results'
# targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-scalar_guessZeroActivityWeights2e3p025__2022-01-21-11-04-05_results'
# targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-scalar_goodCrossHomeoExamp__2022-01-26-15-43-44_results'  # so good
# targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-scalar_goodCrossHomeoExamp__2022-01-26-16-02-50_results'  # 1000 trial version
targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-outer_seed0_explodeDealTest_2022-02-08-10-00-43_results'
# targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-outer-homeo_goodCrossHomeoExamp__2022-01-27-07-30-31_results'

targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-outer_seed2_explodeDealTest_2022-02-10-07-29-19_results'
targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-scalar_seed2_explodeDealTestLowAlpha_2022-02-09-07-28-45_results'


R = Results()
R.init_from_file(targetFile, targetPath)

downSampleFactorForE = 0.025
downSampleFactorForI = 0.1

if R.p['useRule'][:11] == 'cross-homeo':
    R.p['alpha2'] = -1e-12
    R.p['tauPlasticityTrials'] = None
if 'wEEScale' not in R.p:
    R.p['wEEScale'] = None
    R.p['wIEScale'] = None
    R.p['wEIScale'] = None
    R.p['wIIScale'] = None

importantInfoString = 'Name: {}\nEE: {}, IE: {}, EI: {}, II: {}\n tauFR={}, a1={:.4f} pA, a2={:.4f} pA, tauP={}'.format(R.rID,
    R.p['wEEScale'],R.p['wIEScale'],R.p['wEIScale'],R.p['wIIScale'],
    R.p['tauUpFRTrials'], R.p['alpha1'] / pA, R.p['alpha2'] / pA, R.p['tauPlasticityTrials'])

wEEInit = weight_matrix_from_flat_inds_weights(R.p['nExc'], R.p['nExc'], R.preEE, R.posEE, R.wEE_init)
wIEInit = weight_matrix_from_flat_inds_weights(R.p['nExc'], R.p['nInh'], R.preIE, R.posIE, R.wIE_init)
wEIInit = weight_matrix_from_flat_inds_weights(R.p['nInh'], R.p['nExc'], R.preEI, R.posEI, R.wEI_init)
wIIInit = weight_matrix_from_flat_inds_weights(R.p['nInh'], R.p['nInh'], R.preII, R.posII, R.wII_init)

wEEFinal = weight_matrix_from_flat_inds_weights(R.p['nExc'], R.p['nExc'], R.preEE, R.posEE, R.wEE_final)
wIEFinal = weight_matrix_from_flat_inds_weights(R.p['nExc'], R.p['nInh'], R.preIE, R.posIE, R.wIE_final)
wEIFinal = weight_matrix_from_flat_inds_weights(R.p['nInh'], R.p['nExc'], R.preEI, R.posEI, R.wEI_final)
wIIFinal = weight_matrix_from_flat_inds_weights(R.p['nInh'], R.p['nInh'], R.preII, R.posII, R.wII_final)

wFullInit = np.block([[wEEInit, wIEInit], [-wEIInit, -wIIInit]])
wFullFinal = np.block([[wEEFinal, wIEFinal], [-wEIFinal, -wIIFinal]])

# get summed presynaptic input to each unit

wEEInitP = np.nansum(wEEInit, 0)
wIEInitP = np.nansum(wIEInit, 0)
wEIInitP = np.nansum(wEIInit, 0)
wIIInitP = np.nansum(wIIInit, 0)

wEEFinalP = np.nansum(wEEFinal, 0)
wIEFinalP = np.nansum(wIEFinal, 0)
wEIFinalP = np.nansum(wEIFinal, 0)
wIIFinalP = np.nansum(wIIFinal, 0)

downSampleNForE = int(downSampleFactorForE * R.p['nExc'])
downSampleNForI = int(downSampleFactorForI * R.p['nInh'])

downSampleE = np.random.choice(R.p['nExc'], size=downSampleNForE, replace=False)
downSampleI = np.random.choice(R.p['nInh'], size=downSampleNForI, replace=False)

In [6]:
wEEFinal.shape  # (pre, post)

(1600, 1600)

In [7]:
print('wEE', np.nanmean(wEEFinal))
print('wIE', np.nanmean(wIEFinal))
print('wEI', np.nanmean(wEIFinal))
print('wII', np.nanmean(wIIFinal))

wEE 86.21643
wIE 91.918365
wEI 428.10696
wII 307.18378


In [8]:
print('totEE: {:.2f} +/- {:.2f}'.format(wEEFinalP.mean(), wEEFinalP.std()))
print('totEI: {:.2f} +/- {:.2f}'.format(wEIFinalP.mean(), wEIFinalP.std()))
print('totIE: {:.2f} +/- {:.2f}'.format(wIEFinalP.mean(), wIEFinalP.std()))
print('totII: {:.2f} +/- {:.2f}'.format(wIIFinalP.mean(), wIIFinalP.std()))

totEE: 34486.55 +/- 14126.65
totEI: 42810.69 +/- 7961.02
totIE: 36767.34 +/- 6581.25
totII: 30718.38 +/- 6184.35


In [9]:
netCurrentToE = wEEFinalP - wEIFinalP
netCurrentToI = wIEFinalP - wIIFinalP

In [10]:
netCurrentToE.mean()

-8324.137

In [11]:
netCurrentToI.mean()

6048.964

In [12]:
netCurrentToE.std()

21317.479

In [13]:
netCurrentToI.std()

12018.913

In [14]:
plt.close('all')

- idea is to draw a movie of the change in average FR and weights across *trials* between trial n1 and n2
- then draw the movie of a single trial (n2)
- then draw the trials from n2 to n3
- then draw the movie of a single trial (n3)

In [15]:
# define things

clipVeryHighFRs = False
yMax = 40
commonXLims = (0, R.trialUpFRExcUnits.shape[0])
commonTrialRange = np.arange(0, R.trialUpFRExcUnits.shape[0])

if clipVeryHighFRs:
    tmptrialUpFRExcUnits = R.trialUpFRExcUnits.copy()
    tmptrialUpFRInhUnits = R.trialUpFRInhUnits.copy()
    tmptrialUpFRExcUnits[R.trialUpFRExcUnits > R.p['maxAllowedFRExc']] = np.nan
    tmptrialUpFRInhUnits[R.trialUpFRInhUnits > R.p['maxAllowedFRInh']] = np.nan
else:
    tmptrialUpFRExcUnits = R.trialUpFRExcUnits.copy()
    tmptrialUpFRInhUnits = R.trialUpFRInhUnits.copy()
    
# for useTrialInd in range(1, len(R.p['saveTrials'])):
useTrialInd = 9
    
useTrialRange = np.arange(0, R.p['saveTrials'][useTrialInd])

trialwEE = R.trialwEE[useTrialRange]
trialwIE = R.trialwIE[useTrialRange]
trialwEI = R.trialwEI[useTrialRange]
trialwII = R.trialwII[useTrialRange]

#     FRExcUnits = tmptrialUpFRExcUnits[useTrialRange, downSampleE]
#     FRInhUnits = tmptrialUpFRInhUnits[useTrialRange, downSampleI]
FRExcUnits = tmptrialUpFRExcUnits[useTrialRange, :][:, downSampleE]
FRInhUnits = tmptrialUpFRInhUnits[useTrialRange, :][:, downSampleI]

trialUpFRExc = R.trialUpFRExc[useTrialRange]
trialUpFRInh = R.trialUpFRInh[useTrialRange]

In [242]:
# FR and weight convergence line graph - static drawing equivalent

In [16]:
fontProp = fm.FontProperties(fname='C:\\Users\\mikejseay\\anaconda3\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\otf\\MYRIADPRO-REGULAR.otf')
# fontPropSmall = fm.FontProperties(fname='C:\\Users\\mikejseay\\anaconda3\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\otf\\MYRIADPRO-REGULAR.otf')
# fontPropBig = fm.FontProperties(fname='C:\\Users\\mikejseay\\anaconda3\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\otf\\MYRIADPRO-REGULAR.otf')

In [17]:
fontProp.set_size(24)
# fontPropSmall.set_size(14)
# fontPropBig.set_size(14)

In [20]:
startTrialInd = 500
endTrialInd = 5999
movAvgWidth = 5
removeFrame = True
removeAxis = False
doSave = False

plt.close('all')
f, ax = plt.subplots(2, 1, num=1, sharex=True, figsize=(6, 10), frameon=False)

if movAvgWidth:
    useFRExc = moving_average(R.trialUpFRExc[startTrialInd:endTrialInd], movAvgWidth)
    useFRInh = moving_average(R.trialUpFRInh[startTrialInd:endTrialInd], movAvgWidth)
else:
    useFRExc = R.trialUpFRExc[startTrialInd:endTrialInd]
    useFRInh = R.trialUpFRInh[startTrialInd:endTrialInd]

ax[0].plot(useFRExc, label='E', color='g', alpha=1, lw=1)
ax[0].plot(useFRInh, label='I', color='r', alpha=1, lw=1)
ax[0].legend(frameon=False, prop=fontProp)
ax[0].hlines(R.p['setUpFRExc'], 0, len(R.trialUpFRExc[startTrialInd:endTrialInd]), ls='--', color='g')
ax[0].hlines(R.p['setUpFRInh'], 0, len(R.trialUpFRInh[startTrialInd:endTrialInd]), ls='--', color='r')
ax[0].set_ylabel('Firing Rate (Hz)', fontproperties=fontProp)
ax[0].set_ylim(0, 25)

ax[1].plot(R.trialwEE[startTrialInd:endTrialInd], label='wEE', color='cyan')
ax[1].plot(R.trialwIE[startTrialInd:endTrialInd], label='wIE', color='purple')
ax[1].plot(R.trialwEI[startTrialInd:endTrialInd], label='wEI', color='cyan', ls='--')
ax[1].plot(R.trialwII[startTrialInd:endTrialInd], label='wII', color='purple', ls='--')
ax[1].legend(frameon=False, prop=fontProp)
ax[1].set_xlabel('Trial #', fontproperties=fontProp)
ax[1].set_ylabel('Weight (pA)', fontproperties=fontProp)
ax[1].set_ylim(0, ax[1].get_ylim()[1])

for axr in ax:  # christ
    for xtl in axr.get_xticklabels():
        xtl.set_fontproperties(fontProp)
    for ytl in axr.get_yticklabels():
        ytl.set_fontproperties(fontProp)

if removeFrame:
    for axr in ax:
        axr.spines['top'].set_visible(False)
        axr.spines['right'].set_visible(False)

if removeAxis:
    for axr in ax:
        axr.get_xaxis().set_ticks([])
        axr.get_yaxis().set_ticks([])
        axr.axis('off')
        
if doSave:
    saveFileName = 'figs/' + R.rID + '_FRweights.pdf'
    f.savefig(saveFileName, transparent=True)

In [21]:
R.p['saveTrials']

array([  -1,   99,  199,  299,  399,  499,  599,  699,  799,  899,  999,
       1099, 1199, 1299, 1399, 1499, 1599, 1699, 1799, 1899, 1999, 2099,
       2199, 2299, 2399, 2499, 2599, 2699, 2799, 2899, 2999, 3099, 3199,
       3299, 3399, 3499, 3599, 3699, 3799, 3899, 3999, 4099, 4199, 4299,
       4399, 4499, 4599, 4699, 4799, 4899, 4999, 5099, 5199, 5299, 5399,
       5499, 5599, 5699, 5799, 5899])

# plot with voltage, etc, static equivalent (single trial)

In [57]:
# 0, 9, 15

# plt.close('all')
useTrialInd = 27  #24, 27
doSave = False
removeAxis = True
actualTrialIndex = R.p['saveTrials'][useTrialInd] + 1
frameMult = int(R.p['downSampleVoltageTo'] / R.p['dt'])
useUnitInd = 0
decreaseVoltageBy = 70
FRDT = 0.01  # seconds
voltageYLims = (-90, 10)
downsampleRaster = False
targetDisplayedExcUnits = 160
targetDisplayedInhUnits = 40

commonXLims = (0, R.p['duration'] / second)
voltageT = R.selectTrialT[1:]

voltageE = R.selectTrialVExc[useTrialInd, :] - decreaseVoltageBy
vSpikeTimesE = R.selectTrialSpikeExcT[useTrialInd][R.selectTrialSpikeExcI[useTrialInd] == useUnitInd]
vSpikeTimesIndE = (vSpikeTimesE / R.p['dt'] * second / frameMult).astype(int)
voltageE[vSpikeTimesIndE] = 0

voltageI = R.selectTrialVInh[useTrialInd, :] - decreaseVoltageBy
vSpikeTimesI = R.selectTrialSpikeInhT[useTrialInd][R.selectTrialSpikeInhI[useTrialInd] == useUnitInd]
vSpikeTimesIndI = (vSpikeTimesI / R.p['dt'] * second / frameMult).astype(int)
voltageI[vSpikeTimesIndI] = 0

FRT = np.arange(FRDT / 2, R.p['duration'] / second - FRDT / 2, FRDT)
FRE = R.selectTrialFRExc[useTrialInd, :]
FRI = R.selectTrialFRInh[useTrialInd, :]

spikeMonExcT = R.selectTrialSpikeExcT[useTrialInd]
spikeMonExcI = R.selectTrialSpikeExcI[useTrialInd]
spikeMonInhT = R.selectTrialSpikeInhT[useTrialInd]
spikeMonInhI = R.selectTrialSpikeInhI[useTrialInd]

if downsampleRaster:
    downSampleE = np.random.choice(R.p['nExc'], size=targetDisplayedExcUnits, replace=False)
    downSampleI = np.random.choice(R.p['nInh'], size=targetDisplayedInhUnits, replace=False)
    matchingEUnitsBool = np.isin(spikeMonExcI, downSampleE)
    matchingIUnitsBool = np.isin(spikeMonInhI, downSampleI)
    DownSampleERev = np.full((downSampleE.max() + 1,), np.nan)
    DownSampleERev[downSampleE] = np.arange(downSampleE.size)
    DownSampleIRev = np.full((downSampleI.max() + 1,), np.nan)
    DownSampleIRev[downSampleI] = np.arange(downSampleI.size)
    spikesExcT = spikeMonExcT[matchingEUnitsBool]
    spikesExcI = DownSampleERev[spikeMonExcI[matchingEUnitsBool].astype(int)]
    spikesInhT = spikeMonInhT[matchingIUnitsBool]
    spikesInhI = targetDisplayedExcUnits + DownSampleIRev[spikeMonInhI[matchingIUnitsBool].astype(int)]
else:
    spikesExcT = spikeMonExcT
    spikesExcI = spikeMonExcI
    spikesInhT = spikeMonInhT
    spikesInhI = R.p['nExc'] + spikeMonInhI

fig1, ax1 = plt.subplots(4, 1, figsize=(5, 9), gridspec_kw={'height_ratios': [2, 1, 1, 1]}, sharex=True)

srAx, frAx, veAx, viAx = ax1

srAx.set(xlim=commonXLims, ylim=(0, 200), ylabel='Unit Index')      # spike raster
frAx.set(xlim=commonXLims, ylim=(0, 30), ylabel='Firing Rate (Hz)')       # FR
veAx.set(xlim=commonXLims, ylim=voltageYLims, ylabel='mV')  # voltageE
viAx.set(xlim=commonXLims, ylim=voltageYLims, ylabel='mV', xlabel='Time (s)')  # voltageI

if removeAxis:
    for ax in ax1:
        ax.axis('off')
        ax.get_xaxis().set_ticks([])
        ax.get_yaxis().set_ticks([])

srHandleE, = srAx.plot([], [], c='g', ls='', marker='.', markersize=1)
srHandleI, = srAx.plot([], [], c='r', ls='', marker='.', markersize=1)

frHandleE, = frAx.plot([], [], c='g', label='Exc', alpha=0.5)
frHandleI, = frAx.plot([], [], c='r', label='Inh', alpha=0.5)

vHandleE, = veAx.plot([], [], c='g', lw=.3)
vHandleI, = viAx.plot([], [], c='r', lw=.3)

vHandleE.set_data(voltageT, voltageE)
vHandleI.set_data(voltageT, voltageI)

frHandleE.set_data(FRT, FRE)
frHandleI.set_data(FRT, FRI)
srHandleE.set_data(spikesExcT, spikesExcI)
srHandleI.set_data(spikesInhT, spikesInhI)

fig1.suptitle('Trial {}'.format(actualTrialIndex))

if doSave:
    saveFileName = 'figs/' + R.rID + '_trial{}'.format(actualTrialIndex) + '.pdf'
    fig1.savefig(saveFileName, transparent=True)

#### only other things i can think of:
- a histogram of firing rates / weights
- the weight matrix
- E/I balance for each unit
- up duration over trials

In [218]:
# duration

endTrialInd = 2500
movAvgWidth = 9
removeFrame = True
doSave = True

f, ax = plt.subplots(figsize=(6, 3))
ax.plot(moving_average(R.trialUpDur[:endTrialInd], movAvgWidth), color='k')
ax.set_xlabel('Trial #', fontproperties=fontProp)
ax.set_ylabel('Response Duration (s)', fontproperties=fontProp)
for xtl in ax.get_xticklabels():
    xtl.set_fontproperties(fontProp)
for ytl in ax.get_yticklabels():
    ytl.set_fontproperties(fontProp)

if removeFrame:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
if doSave:
    saveFileName = 'figs/' + R.rID + '_upDur.pdf'
    f.savefig(saveFileName, transparent=True)

In [234]:
plt.close('all')

In [235]:
# histogram of FRs

finalTrials = 1000
nBins = 40
excFRs = R.trialUpFRExcUnits[-finalTrials:, :].mean(0)
inhFRs = R.trialUpFRInhUnits[-finalTrials:, :].mean(0)
removeFrame = True
doSave = True

# minFR = np.fmin(excFRs.min(), inhFRs.min())
# maxFR = np.fmax(excFRs.max(), inhFRs.max())

minFR = 0
maxFR = 30

frBins = np.linspace(minFR, maxFR, nBins)

f, ax = plt.subplots(figsize=(9, 5))

ax.hist(R.trialUpFRExcUnits[-1, :], frBins, color='g', density=True, histtype='step', label='E')
ax.hist(R.trialUpFRInhUnits[-1, :], frBins, color='r', density=True, histtype='step', label='I')
ax.set_xlabel('FR (Hz)', fontproperties=fontProp)
ax.set_ylabel('# of occurences', fontproperties=fontProp)
for xtl in ax.get_xticklabels():
    xtl.set_fontproperties(fontProp)
for ytl in ax.get_yticklabels():
    ytl.set_fontproperties(fontProp)
ax.legend(frameon=False, prop=fontProp)

if removeFrame:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
if doSave:
    saveFileName = 'figs/' + R.rID + '_FRHist.pdf'
    f.savefig(saveFileName, transparent=True)

In [237]:
np.std(excFRs) / excFRs.mean()

0.7846408470767985

In [238]:
np.std(inhFRs) / inhFRs.mean()

0.836804853749558

In [59]:
# initial and final weight matrices

In [ ]:
# 2D weight progression (scatterplot)

In [9]:
# plot with voltage, etc, video equivalent (single trial)

frameMult = int(R.p['downSampleVoltageTo'] / R.p['dt'])
useUnitInd = 0
decreaseVoltageBy = 70
FRDT = 0.01  # seconds
voltageYLims = (-90, 10)
targetDisplayedExcUnits = 100
targetDisplayedInhUnits = 100

commonXLims = (0, R.p['duration'] / second)
voltageT = R.selectTrialT[1:]

voltageE = R.selectTrialVExc[useTrialInd, :] - decreaseVoltageBy
vSpikeTimesE = R.selectTrialSpikeExcT[useTrialInd][R.selectTrialSpikeExcI[useTrialInd] == useUnitInd]
vSpikeTimesIndE = (vSpikeTimesE / R.p['dt'] * second / frameMult).astype(int)
voltageE[vSpikeTimesIndE] = 0

voltageI = R.selectTrialVInh[useTrialInd, :] - decreaseVoltageBy
vSpikeTimesI = R.selectTrialSpikeInhT[useTrialInd][R.selectTrialSpikeInhI[useTrialInd] == useUnitInd]
vSpikeTimesIndI = (vSpikeTimesI / R.p['dt'] * second / frameMult).astype(int)
voltageI[vSpikeTimesIndI] = 0

FRT = np.arange(FRDT / 2, R.p['duration'] / second - FRDT / 2, FRDT)
FRE = R.selectTrialFRExc[useTrialInd, :]
FRI = R.selectTrialFRInh[useTrialInd, :]

spikeMonExcT = R.selectTrialSpikeExcT[useTrialInd]
spikeMonExcI = R.selectTrialSpikeExcI[useTrialInd]
spikeMonInhT = R.selectTrialSpikeInhT[useTrialInd]
spikeMonInhI = R.selectTrialSpikeInhI[useTrialInd]

downSampleE = np.random.choice(R.p['nExc'], size=targetDisplayedExcUnits, replace=False)
downSampleI = np.random.choice(R.p['nInh'], size=targetDisplayedInhUnits, replace=False)
matchingEUnitsBool = np.isin(spikeMonExcI, downSampleE)
matchingIUnitsBool = np.isin(spikeMonInhI, downSampleI)
DownSampleERev = np.full((downSampleE.max() + 1,), np.nan)
DownSampleERev[downSampleE] = np.arange(downSampleE.size)
DownSampleIRev = np.full((downSampleI.max() + 1,), np.nan)
DownSampleIRev[downSampleI] = np.arange(downSampleI.size)
spikesExcT = spikeMonExcT[matchingEUnitsBool]
spikesExcI = DownSampleERev[spikeMonExcI[matchingEUnitsBool].astype(int)]
spikesInhT = spikeMonInhT[matchingIUnitsBool]
spikesInhI = targetDisplayedExcUnits + DownSampleIRev[spikeMonInhI[matchingIUnitsBool].astype(int)]

fig1, ax1 = plt.subplots(4, 1, num=1, figsize=(5, 9), gridspec_kw={'height_ratios': [2, 1, 1, 1]}, sharex=True)

srAx, frAx, veAx, viAx = ax1

srAx.set(xlim=commonXLims, ylim=(0, 200), ylabel='Unit Index')      # spike raster
frAx.set(xlim=commonXLims, ylim=(0, 30), ylabel='Firing Rate (Hz)')       # FR
veAx.set(xlim=commonXLims, ylim=voltageYLims, ylabel='mV')  # voltageE
viAx.set(xlim=commonXLims, ylim=voltageYLims, ylabel='mV', xlabel='Time (s)')  # voltageI

srHandleE, = srAx.plot([], [], c='g', ls='', marker='.', markersize=1)
srHandleI, = srAx.plot([], [], c='r', ls='', marker='.', markersize=1)

frHandleE, = frAx.plot([], [], c='g', label='Exc', alpha=0.5)
frHandleI, = frAx.plot([], [], c='r', label='Inh', alpha=0.5)

vHandleE, = veAx.plot([], [], c='g', lw=.3)
vHandleI, = viAx.plot([], [], c='r', lw=.3)

def init():
    srHandleE.set_data([], [])
    srHandleI.set_data([], [])
    frHandleE.set_data([], [])
    frHandleI.set_data([], [])
    vHandleE.set_data([], [])
    vHandleI.set_data([], [])

    print('initalized animation')

    return srHandleE, srHandleI, frHandleE, frHandleI, vHandleE, vHandleI, 

def animate(i):
    vHandleE.set_data(voltageT[:i], voltageE[:i])
    vHandleI.set_data(voltageT[:i], voltageI[:i])

    fri = int(i / 10)
    frHandleE.set_data(FRT[:fri], FRE[:fri])
    frHandleI.set_data(FRT[:fri], FRI[:fri])

    currentTime = voltageT[i]
    sBoolE = spikesExcT <= currentTime
    sBoolI = spikesInhT <= currentTime
    srHandleE.set_data(spikesExcT[sBoolE], spikesExcI[sBoolE])
    srHandleI.set_data(spikesInhT[sBoolI], spikesInhI[sBoolI])

    print('animated frame', i)

    return srHandleE, srHandleI, frHandleE, frHandleI, vHandleE, vHandleI, 

useFPS = 30
totalFrames = int(np.round((voltageT[-1] * second) * useFPS * Hz))
useInterval = int(np.round(voltageT.size / totalFrames))
framesToSave = np.arange(0, voltageT.size, useInterval)

anim = FuncAnimation(fig1, animate, init_func=init,
                     frames=framesToSave, blit=True, repeat=False)
# anim.save('trial' + str(useTrialInd) + '.gif')

Writer = writers['ffmpeg']
writer = Writer(fps=useFPS)
anim.save('trial' + str(R.p['saveTrials'][useTrialInd] + 1) + '.mp4', writer=writer)
fig1.clear()

initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
animated frame 231
animated frame 264
animated frame 297
animated frame 330
animated frame 363
animated frame 396
animated frame 429
animated frame 462
animated frame 495
animated frame 528
animated frame 561
animated frame 594
animated frame 627
animated frame 660
animated frame 693
animated frame 726
animated frame 759
animated frame 792
animated frame 825
animated frame 858
animated frame 891
animated frame 924
animated frame 957
animated frame 990
animated frame 1023
animated frame 1056
animated frame 1089
animated frame 1122
animated frame 1155
animated frame 1188
animated frame 1221
animated frame 1254
animated frame 1287
animated frame 1320
animated frame 1353
animated frame 1386
animated frame 1419
animated frame 1452
animated frame 1485
initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
ani

animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
animated frame 231
animated frame 264
animated frame 297
animated frame 330
animated frame 363
animated frame 396
animated frame 429
animated frame 462
animated frame 495
animated frame 528
animated frame 561
animated frame 594
animated frame 627
animated frame 660
animated frame 693
animated frame 726
animated frame 759
animated frame 792
animated frame 825
animated frame 858
animated frame 891
animated frame 924
animated frame 957
animated frame 990
animated frame 1023
animated frame 1056
animated frame 1089
animated frame 1122
animated frame 1155
animated frame 1188
animated frame 1221
animated frame 1254
animated frame 1287
animated frame 1320
animated frame 1353
animated frame 1386
animated frame 1419
animated frame 1452
animated frame 1485
initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
an

TypeError: 'NoneType' object is not subscriptable

initalized animation
initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
animated frame 231
animated frame 264
animated frame 297
animated frame 330
animated frame 363
animated frame 396
animated frame 429
animated frame 462
animated frame 495
animated frame 528
animated frame 561
animated frame 594
animated frame 627
animated frame 660
animated frame 693
animated frame 726
animated frame 759
animated frame 792
animated frame 825
animated frame 858
animated frame 891
animated frame 924
animated frame 957
animated frame 990
animated frame 1023
animated frame 1056
animated frame 1089
animated frame 1122
animated frame 1155
animated frame 1188
animated frame 1221
animated frame 1254
animated frame 1287
animated frame 1320
animated frame 1353
animated frame 1386
animated frame 1419
animated frame 1452
animated frame 1485


In [9]:
# plot with voltage, etc, video equivalent (multiple trials)

frameMult = int(R.p['downSampleVoltageTo'] / R.p['dt'])
useUnitInd = 0
decreaseVoltageBy = 70
FRDT = 0.01  # seconds
voltageYLims = (-90, 10)
targetDisplayedExcUnits = 100
targetDisplayedInhUnits = 100

commonXLims = (0, R.p['duration'] / second)
voltageT = R.selectTrialT[1:]

for useTrialInd in range(len(R.p['saveTrials'])):

    voltageE = R.selectTrialVExc[useTrialInd, :] - decreaseVoltageBy
    vSpikeTimesE = R.selectTrialSpikeExcT[useTrialInd][R.selectTrialSpikeExcI[useTrialInd] == useUnitInd]
    vSpikeTimesIndE = (vSpikeTimesE / R.p['dt'] * second / frameMult).astype(int)
    voltageE[vSpikeTimesIndE] = 0

    voltageI = R.selectTrialVInh[useTrialInd, :] - decreaseVoltageBy
    vSpikeTimesI = R.selectTrialSpikeInhT[useTrialInd][R.selectTrialSpikeInhI[useTrialInd] == useUnitInd]
    vSpikeTimesIndI = (vSpikeTimesI / R.p['dt'] * second / frameMult).astype(int)
    voltageI[vSpikeTimesIndI] = 0

    FRT = np.arange(FRDT / 2, R.p['duration'] / second - FRDT / 2, FRDT)
    FRE = R.selectTrialFRExc[useTrialInd, :]
    FRI = R.selectTrialFRInh[useTrialInd, :]

    spikeMonExcT = R.selectTrialSpikeExcT[useTrialInd]
    spikeMonExcI = R.selectTrialSpikeExcI[useTrialInd]
    spikeMonInhT = R.selectTrialSpikeInhT[useTrialInd]
    spikeMonInhI = R.selectTrialSpikeInhI[useTrialInd]

    downSampleE = np.random.choice(R.p['nExc'], size=targetDisplayedExcUnits, replace=False)
    downSampleI = np.random.choice(R.p['nInh'], size=targetDisplayedInhUnits, replace=False)
    matchingEUnitsBool = np.isin(spikeMonExcI, downSampleE)
    matchingIUnitsBool = np.isin(spikeMonInhI, downSampleI)
    DownSampleERev = np.full((downSampleE.max() + 1,), np.nan)
    DownSampleERev[downSampleE] = np.arange(downSampleE.size)
    DownSampleIRev = np.full((downSampleI.max() + 1,), np.nan)
    DownSampleIRev[downSampleI] = np.arange(downSampleI.size)
    spikesExcT = spikeMonExcT[matchingEUnitsBool]
    spikesExcI = DownSampleERev[spikeMonExcI[matchingEUnitsBool].astype(int)]
    spikesInhT = spikeMonInhT[matchingIUnitsBool]
    spikesInhI = targetDisplayedExcUnits + DownSampleIRev[spikeMonInhI[matchingIUnitsBool].astype(int)]

    fig1, ax1 = plt.subplots(4, 1, num=1, figsize=(5, 9), gridspec_kw={'height_ratios': [2, 1, 1, 1]}, sharex=True)

    srAx, frAx, veAx, viAx = ax1

    srAx.set(xlim=commonXLims, ylim=(0, 200), ylabel='Unit Index')      # spike raster
    frAx.set(xlim=commonXLims, ylim=(0, 30), ylabel='Firing Rate (Hz)')       # FR
    veAx.set(xlim=commonXLims, ylim=voltageYLims, ylabel='mV')  # voltageE
    viAx.set(xlim=commonXLims, ylim=voltageYLims, ylabel='mV', xlabel='Time (s)')  # voltageI

    srHandleE, = srAx.plot([], [], c='g', ls='', marker='.', markersize=1)
    srHandleI, = srAx.plot([], [], c='r', ls='', marker='.', markersize=1)

    frHandleE, = frAx.plot([], [], c='g', label='Exc', alpha=0.5)
    frHandleI, = frAx.plot([], [], c='r', label='Inh', alpha=0.5)

    vHandleE, = veAx.plot([], [], c='g', lw=.3)
    vHandleI, = viAx.plot([], [], c='r', lw=.3)

    def init():
        srHandleE.set_data([], [])
        srHandleI.set_data([], [])
        frHandleE.set_data([], [])
        frHandleI.set_data([], [])
        vHandleE.set_data([], [])
        vHandleI.set_data([], [])

        print('initalized animation')

        return srHandleE, srHandleI, frHandleE, frHandleI, vHandleE, vHandleI, 

    def animate(i):
        vHandleE.set_data(voltageT[:i], voltageE[:i])
        vHandleI.set_data(voltageT[:i], voltageI[:i])

        fri = int(i / 10)
        frHandleE.set_data(FRT[:fri], FRE[:fri])
        frHandleI.set_data(FRT[:fri], FRI[:fri])

        currentTime = voltageT[i]
        sBoolE = spikesExcT <= currentTime
        sBoolI = spikesInhT <= currentTime
        srHandleE.set_data(spikesExcT[sBoolE], spikesExcI[sBoolE])
        srHandleI.set_data(spikesInhT[sBoolI], spikesInhI[sBoolI])

        print('animated frame', i)

        return srHandleE, srHandleI, frHandleE, frHandleI, vHandleE, vHandleI, 

    useFPS = 30
    totalFrames = int(np.round((voltageT[-1] * second) * useFPS * Hz))
    useInterval = int(np.round(voltageT.size / totalFrames))
    framesToSave = np.arange(0, voltageT.size, useInterval)

    anim = FuncAnimation(fig1, animate, init_func=init,
                         frames=framesToSave, blit=True, repeat=False)
    # anim.save('trial' + str(useTrialInd) + '.gif')

    Writer = writers['ffmpeg']
    writer = Writer(fps=useFPS)
    anim.save('trial' + str(R.p['saveTrials'][useTrialInd] + 1) + '.mp4', writer=writer)
    fig1.clear()

initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
animated frame 231
animated frame 264
animated frame 297
animated frame 330
animated frame 363
animated frame 396
animated frame 429
animated frame 462
animated frame 495
animated frame 528
animated frame 561
animated frame 594
animated frame 627
animated frame 660
animated frame 693
animated frame 726
animated frame 759
animated frame 792
animated frame 825
animated frame 858
animated frame 891
animated frame 924
animated frame 957
animated frame 990
animated frame 1023
animated frame 1056
animated frame 1089
animated frame 1122
animated frame 1155
animated frame 1188
animated frame 1221
animated frame 1254
animated frame 1287
animated frame 1320
animated frame 1353
animated frame 1386
animated frame 1419
animated frame 1452
animated frame 1485
initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
ani

animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
animated frame 231
animated frame 264
animated frame 297
animated frame 330
animated frame 363
animated frame 396
animated frame 429
animated frame 462
animated frame 495
animated frame 528
animated frame 561
animated frame 594
animated frame 627
animated frame 660
animated frame 693
animated frame 726
animated frame 759
animated frame 792
animated frame 825
animated frame 858
animated frame 891
animated frame 924
animated frame 957
animated frame 990
animated frame 1023
animated frame 1056
animated frame 1089
animated frame 1122
animated frame 1155
animated frame 1188
animated frame 1221
animated frame 1254
animated frame 1287
animated frame 1320
animated frame 1353
animated frame 1386
animated frame 1419
animated frame 1452
animated frame 1485
initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
an

TypeError: 'NoneType' object is not subscriptable

initalized animation
initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
animated frame 231
animated frame 264
animated frame 297
animated frame 330
animated frame 363
animated frame 396
animated frame 429
animated frame 462
animated frame 495
animated frame 528
animated frame 561
animated frame 594
animated frame 627
animated frame 660
animated frame 693
animated frame 726
animated frame 759
animated frame 792
animated frame 825
animated frame 858
animated frame 891
animated frame 924
animated frame 957
animated frame 990
animated frame 1023
animated frame 1056
animated frame 1089
animated frame 1122
animated frame 1155
animated frame 1188
animated frame 1221
animated frame 1254
animated frame 1287
animated frame 1320
animated frame 1353
animated frame 1386
animated frame 1419
animated frame 1452
animated frame 1485
